In [3]:
config_path = '/lab/repos/celldom/config/experiment/experiment_example_G3.yaml'
data_dir = '/lab/data/celldom/output/20180923-G3-full'
import os
import os.path as osp
import numpy as np
import pandas as pd
from celldom.config import experiment_config
import celldom
import plotnine as pn
import matplotlib.pyplot as plt
from celldom.core import cytometry
from celldom.core import modeling
from celldom.execute import analysis

In [5]:
config = experiment_config.ExperimentConfig(config_path)
store = cytometry.get_readonly_datastore(data_dir)
df = store.get('apartment')
df = analysis.add_experiment_date_groups(df, config.experimental_condition_fields)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136729 entries, 0 to 136728
Data columns (total 40 columns):
acq_apartment             136729 non-null object
acq_cohort                136729 non-null object
acq_color                 136729 non-null object
acq_condition             136729 non-null object
acq_datetime              136729 non-null datetime64[ns]
acq_hours                 136729 non-null object
acq_id                    136729 non-null object
acq_processed_at          136729 non-null datetime64[ns]
acq_pump                  136729 non-null object
acq_street                136729 non-null object
apt_id                    136729 non-null int64
apt_image_height          136729 non-null int64
apt_image_width           136729 non-null int64
apt_num                   136729 non-null object
apt_num_digit_scores      136729 non-null object
cell_area_mean            120114 non-null float64
cell_area_p10             120114 non-null float64
cell_area_p50             120114 non-null

In [9]:
X = df.groupby(config.experimental_condition_fields + ['apt_num', 'st_num', 'elapsed_hours_group'])['cell_count'].median().reset_index()
X = X.set_index(config.experimental_condition_fields + ['apt_num', 'st_num'])
X.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 122455 entries, (Elveflow, Orange, 1, 0.1uM, 00, 00) to (gravity, Yellow, 3, 0.3uM, 57, 14)
Data columns (total 2 columns):
elapsed_hours_group    122455 non-null float64
cell_count             122455 non-null float64
dtypes: float64(2)
memory usage: 2.6+ MB


In [25]:
X.head(10)

elapsed_hours_group  \
acq_pump acq_color acq_condition acq_cohort apt_num st_num                        
Elveflow Orange    1             0.1uM      00      00                 0.000000   
                                                    00                21.650000   
                                                    00                74.316667   
                                                    00               100.316667   
                                                    00               121.350000   
                                                    00               150.183333   
                                                    00               166.666667   
                                                    00               197.650000   
                                                    01                 0.000000   
                                                    01                21.650000   

                                                            cell_count  
acq_pump acq_color acq_condition acq_cohort apt_num st_num              
Elveflow Orange    1             0.1uM      00      00             2.0  
                                                    00             9.5  
                                                    00            14.0  
                                                    00            30.0  
                                                    00            57.0  
                                                    00           102.0  
                                                    00           137.0  
                                                    00            73.0  
                                                    01             3.0  
                                                    01             4.0

In [24]:
from statsmodels.miscmodels.count import PoissonZiGMLE

def run_model(g):
    if len(g) <= 1:
        return np.array([np.nan, np.nan])
    est = PoissonZiGMLE(g['cell_count'], exog=g['elapsed_hours_group'])
    res = est.fit(disp=0)
    return res.params
dfres = X.reset_index().groupby(X.index).apply(run_model)

/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.5/dist-packages/s

KeyboardInterrupt: 

In [12]:
from statsmodels.miscmodels.count import PoissonZiGMLE
est = PoissonZiGMLE(X['cell_count'], exog=X['elapsed_hours_group'])

In [15]:
res = est.fit()

Optimization terminated successfully.
         Current function value: 4.393091
         Iterations: 87
         Function evaluations: 163


In [17]:
res.params

array([0.01829093, 2.68892542])

In [18]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            PoissonZiGMLE Results                             
==============================================================================
Dep. Variable:             cell_count   Log-Likelihood:            -5.3796e+05
Model:                  PoissonZiGMLE   AIC:                         1.076e+06
Method:            Maximum Likelihood   BIC:                         1.076e+06
Date:                Tue, 25 Sep 2018                                         
Time:                        00:30:17                                         
No. Observations:              122455                                         
Df Residuals:                  122454                                         
Df Model:                           0                                         
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
elapsed_hours_group     0.0183   1.24e-05   1469.265      0.000       0.018       0.018
zi                      2.6889      0.014    195.416      0.000       2.662       2.716
=======================================================================================
"""